In [ ]:
# Extracting text from the doc and preprocessing using preprocessor from HS
# Creating a prompt template
# Loading a retriver and inmemory store
# Created a , HS pipeline - using prompt node with mistral, retriver
# Running RAG pipeline on the list of questions

In [1]:
# !pip install farm-haystack[colab] PyPDF2 python-dotenv -q
# !pip uninstall -y farm-haystack haystack-ai -q
# !pip install haystack-ai -q
!pip install PyPDF2 python-dotenv -q

In [2]:
import os
os.environ["GROQ_API_KEY"]="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"

In [5]:
import PyPDF2
from pprint import pprint
from getpass import getpass
from haystack import Document
from haystack import Pipeline
from haystack.nodes import BM25Retriever # requires farm-haystack
# https://stackoverflow.com/questions/76031413/modulenotfounderror-no-module-named-haystack-nodes
# cannot import name 'PipelineSchemaError' from 'haystack.errors' 
# https://github.com/deepset-ai/haystack/issues/6771 informs farm-haystack is in maintenance mode
# and requests to shift to haystack-ai 2.0 
# review the migration guide here 
from haystack.document_stores import InMemoryDocumentStore
from haystack.nodes import PreProcessor, PromptModel, PromptTemplate, PromptNode

ImportError: cannot import name 'PipelineSchemaError' from 'haystack.errors' (d:\haystackenv\Lib\site-packages\haystack\errors.py)

In [18]:
pdf_file_path = '/content/CUDAThread-IndexingCheatsheet.pdf'
def extract_text_from_pdf(pdf_path):
  text=''
  with open(pdf_path,'rb') as pdf_file:
    pdf_reader = PyPDF2.PdfReader(pdf_file)
    for page_num in range(len(pdf_reader.pages)):
      page = pdf_reader.pages[page_num]
      text += page.extract_text()
      return text
pdf_text = extract_text_from_pdf(pdf_file_path)
doc = Document(content=pdf_text, meta={"pdf_path":pdf_file_path})
docs = [doc]
doc

<Document: {'content': 'CUDA Thread Indexing Cheatsheet If you are a CUDA parallel programmer but sometimes you cannot wrap your head around thread indexing just like me then you are at the right place.  Many problems are naturally described in a flat, linear style mimicking our mental model of C’s memory layout. However, other tasks, especially those encountered in the computational sciences, are naturally embedded in two or three dimensions. For example, image processing tasks typically impose a regular 2D raster over the problem domain while computational fluid dynamics might be most naturally expressed by partitioning a volume over 3D grid.  \n  NVIDIA CUDA Thread Model   \n', 'content_type': 'text', 'score': None, 'meta': {'pdf_path': '/content/CUDAThread-IndexingCheatsheet.pdf'}, 'id_hash_keys': ['content'], 'embedding': None, 'id': 'db6624d7b773dcd2f6a93a97cf8d3c41'}>

In [19]:
processor = PreProcessor(clean_whitespace = True,
                         clean_header_footer = True,
                         clean_empty_lines = True,
                         split_by='word',
                         split_length = 500,
                         split_respect_sentence_boundary= True,
                         split_overlap=0)

processed_docs = processor.process(docs)

Preprocessing: 100%|██████████| 1/1 [00:00<00:00, 522.26docs/s]


In [21]:
len(processed_docs)

1

In [22]:
document_store = InMemoryDocumentStore(use_bm25=True)
document_store.write_documents(processed_docs)
retriver = BM25Retriever(document_store, top_k=2)

Updating BM25 representation...: 100%|██████████| 1/1 [00:00<00:00, 1134.82 docs/s]


In [23]:
qa_template = PromptTemplate(prompt=
      """Using exclusively the information contained in the context, answer only the question asked without adding
      suggestions for possible questions, and respond exclusively in english. If the answer can not be deduced from the
      context ,
      respond:"Not sure because not relevant to the context".
      Context:{join(documents)};
      Question:{query}
      """)

In [8]:
from haystack.components.generators.openai import OpenAIGenerator

# there is some import error in the haystack
# ImportError: cannot import name 'default_from_dict' from 'haystack
# https://github.com/deepset-ai/haystack/issues/6470 tells restart session after pip install
# module 'haystack.components' has no attribute 'output_types' is not found

AttributeError: module 'haystack.components' has no attribute 'output_types'

In [24]:
from haystack.components.generators import OpenAIGenerator

llm = OpenAIGenerator(
    api_key=os.environ["GROQ_API_KEY"],
    api_base_url="https://api.groq.com/openai/v1",
    model="mixtral-8x7b-32768",
    generation_kwargs = {"max_tokens": 512}
)

ImportError: cannot import name 'component' from 'haystack' (/usr/local/lib/python3.10/dist-packages/haystack/__init__.py)

In [10]:
prompt_node = PromptNode(
    api_base="https://api.groq.com/openai/v1",
    model_name_or_path='mixtral-8x7b-32768',
    default_prompt_template=qa_template,
    max_length = 500,
    model_kwargs={"model_max_length":5000}
)

ValueError: Model mixtral-8x7b-32768 is not supported - no matching invocation layer found. Currently supported invocation layers are: [<class 'haystack.nodes.prompt.invocation_layer.open_ai.OpenAIInvocationLayer'>, <class 'haystack.nodes.prompt.invocation_layer.azure_open_ai.AzureOpenAIInvocationLayer'>, <class 'haystack.nodes.prompt.invocation_layer.chatgpt.ChatGPTInvocationLayer'>, <class 'haystack.nodes.prompt.invocation_layer.azure_chatgpt.AzureChatGPTInvocationLayer'>, <class 'haystack.nodes.prompt.invocation_layer.anthropic_claude.AnthropicClaudeInvocationLayer'>, <class 'haystack.nodes.prompt.invocation_layer.cohere.CohereInvocationLayer'>, <class 'haystack.nodes.prompt.invocation_layer.hugging_face.HFLocalInvocationLayer'>, <class 'haystack.nodes.prompt.invocation_layer.hugging_face_inference.HFInferenceEndpointInvocationLayer'>, <class 'haystack.nodes.prompt.invocation_layer.aws_base.AWSBaseInvocationLayer'>, <class 'haystack.nodes.prompt.invocation_layer.amazon_bedrock.Amazo

In [ ]:
rag_pipeline = Pipeline()
rag_pipeline.add_node(component=retriver, name='retriver',inputs=['Query'])
rag_pipeline.add_node(component=prompt_node,name='prompt_node',inputs=['retriver'])

question_list = ['invoice number in the document', 'seller tax id in the document','seller address in the document',
                 'client tax id in the document','IBAN no in the document']
print_answer = lambda out:pprint(out['results'][0].strip())
for question in question_list:
  print_answer(rag_pipeline.run(query=question))



In [ ]:
from haystack import Pipeline
# from haystack.utils import Secret
from haystack.components.fetchers import LinkContentFetcher
from haystack.components.converters import HTMLToDocument
from haystack.components.builders import PromptBuilder
from haystack.components.generators import OpenAIGenerator

fetcher = LinkContentFetcher()
converter = HTMLToDocument()
prompt_template = """
According to the contents of this website:
{% for document in documents %}
  {{document.content}}
{% endfor %}
Answer the given question: {{query}}
Answer:
"""
prompt_builder = PromptBuilder(template=prompt_template)
llm = OpenAIGenerator(
    api_key=GROQ_API_KEY,
    api_base_url="https://api.groq.com/openai/v1",
    model="mixtral-8x7b-32768",
    generation_kwargs = {"max_tokens": 512}
)
pipeline = Pipeline()
pipeline.add_component("fetcher", fetcher)
pipeline.add_component("converter", converter)
pipeline.add_component("prompt", prompt_builder)
pipeline.add_component("llm", llm)

pipeline.connect("fetcher.streams", "converter.sources")
pipeline.connect("converter.documents", "prompt.documents")
pipeline.connect("prompt.prompt", "llm.prompt")

result = pipeline.run({"fetcher": {"urls": ["https://wow.groq.com/why-groq/"]},
              "prompt": {"query": "Why should I use Groq for serving LLMs?"}})

print(result["llm"]["replies"][0])


AttributeError: 'str' object has no attribute 'resolve_value'

In [ ]:
!pip install haystack-ai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 319.5/319.5 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.7/320.7 kB 31.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 7.7 MB/s eta 0:00:00


In [ ]:
from haystack.components.generators.chat import OpenAIChatGenerator
from haystack.dataclasses import ChatMessage
from haystack.utils import Secret

generator = OpenAIChatGenerator(
    api_key=os.environ["GROQ_API_KEY"],
    api_base_url="https://api.groq.com/openai/v1/models",
    model="llama3-8b-8192",
    generation_kwargs = {"max_tokens": 512}
)


messages = []

while True:
    msg = input("Enter your message or Q to exit\n🧑 ")
    if msg=="Q":
        break
    messages.append(ChatMessage.from_user(msg))
    response = generator.run(messages=messages)
    assistant_resp = response['replies'][0]
    print("🤖 "+assistant_resp.content)
    messages.append(assistant_resp)


AttributeError: 'str' object has no attribute 'resolve_value'